<a href="https://colab.research.google.com/github/naomori/codexa_ImageRecognitionForBeginner/blob/master/Chapter4_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4.3 機械学習手法で画像認識

本チャプターでは、ロジスティック回帰を使って、MNISTの分類を行ってみます。

## ロジスティック回帰で画像認識

ロジスティック回帰は 1958 年に統計学者により発明されました。  
ロジスティック回帰は「分類問題に対する確率論的アプローチ」です。  
ロジスティック回帰の目標は、サンプルが特定のカテゴリやクラスに属する「確率」をモデル化することです。

ロジスティック回帰の特徴は以下です。
* シンプルな構造で推論結果の「説明」が比較的簡単
* ハイパーパラメータチューニングが（ほぼ）不要
* **線形性（linear）を仮定**

重要なのは、線形性であり、線形性とは変数と変数の関係が直線的ということを意味します。 

In [1]:
# データ処理のライブラリ
import numpy as np 
import pandas as pd

# データ可視化のライブラリ
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

# 機械学習ライブラリ scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_openml

In [2]:
# MNIST データセットの読み込み
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

# 訓練/テストセットへ分割
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2, random_state=42) 

In [3]:
# 訓練データの最初の1万件のみへ縮小
X_train = X_train[:10000]
y_train = y_train[:10000]

# テストデータの最初の1万件のみへ縮小
X_test = X_test[:10000]
y_test = y_test[:10000]

In [4]:
# 正規化
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

ロジスティック回帰には Scikit-learn の　[LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)を使います。  
これを使って、モデルを訓練します。

In [5]:
# ロジスティック回帰 インスタンス生成
logi = LogisticRegression(solver='lbfgs', multi_class='auto', random_state=42)

# モデル訓練
logi.fit(X_train, y_train)

C:\Users\4009953\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

モデルの訓練が終了したら、テストデータを使って評価します。  
訓練済みモデルへテストデータを入力して、推測結果を混同行列（Confusion Matrix）で出力します。

#### 混同行列（Confusion Matrix）
とは、クラス分類の結果をまとめた表のことです。

陽性のサンプルのうち、何個が正しく陽性と判定され、何個が誤って陰性と判定されたか、  
といったことを分かりやすくまとめるために用います。クロス表の一種です。

In [6]:
# テストデータから推測
logi_pred = logi.predict(X_test)

# 混同行列を表示
print(confusion_matrix(y_test, logi_pred))

[[ 938    0    4    2    2   13    8    3   12    1]
 [   0 1111    5    5    0   12    0    3   12    4]
 [   3   22  843   23   13    3   20   10   24    6]
 [   1    6   29  902    2   43    3   18   19   11]
 [   5    1    8    6  816    2   12    7   15   34]
 [  12    7   10   47   10  796    9    5   32    9]
 [   9    3   15    0    8   18  904    0    3    1]
 [   5    2   19    5    9    5    0  978    1   31]
 [  10   16   18   43    6   33    9    9  809   16]
 [   4    6    3   16   40    4    0   42   15  906]]


正解率を表示してみます。

In [7]:
# 正解率を算出
print(accuracy_score(y_test, logi_pred))

0.9003


# サポートベクターマシンで画像認識

本セクションではサポートベクターマシン（Support Vector Machine: SVM）を使って、MNIST を分類します。  
SVM は回帰・分類ともに使えます。特に分類の推測精度が高いことから多く用いられています。  
カーネル法と呼ばれる手法が特徴的です。  また、計算コストが高いです。
ロジスティック回帰で訓練時間が数秒のものが、SVMでは数分かかります。

SVMでモデル訓練を行います。
Scikit-learn の Support Vector Classifier(SVC) を使います。

In [8]:
# SVM インスタンス生成
svc = SVC(gamma='auto', random_state=42)

# モデル訓練
svc.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

テストデータを使って評価します。

In [10]:
# テストデータから推測
svc_pred = svc.predict(X_test)

# 混同行列を表示
print(confusion_matrix(y_test, svc_pred))

[[ 955    0    5    1    1    6    6    1    8    0]
 [   0 1125    5    7    0    4    1    4    5    1]
 [   8   10  876   10   12    4   18    8   19    2]
 [   2   13   26  903    0   35    6   13   25   11]
 [   3    4    7    0  840    0    6    2    1   43]
 [   6   18    5   41    7  833   10    0   10    7]
 [   7    4    6    1    9   16  914    0    4    0]
 [   2   19   13    1    9    5    0  979    3   24]
 [   7   20   12   40    8   28    9   10  824   11]
 [   8    9    4   14   40    5    0   26    5  925]]


In [11]:
print(accuracy_score(y_test, svc_pred))

0.9174


Logistic Regression と比較して若干 SVM の方が性能が良いみたいです。  
ですが、SVM の方が Logistic Regression より常に優れているとういわけではないです。  
機械学習の手法には特徴があり、その優劣をつけることはできません。

# ランダムフォレストで画像認識

ランダムフォレストは決定木を複数使って、それらを１つの「森」にして、より正確かつ質の高い予測（分類） を行います。
このように、複数の弱いモデル（弱学習器）を１つにまとめる手法を「アンサンブル学習」と呼びます。  
アンサンブル学習は以下の３つに分類されます。
1. Bagging
2. Boosting
3. Stacking

ランダムフォレストは、Bagging に属する学習手法です。

[RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)を使います。

In [12]:
# ランダムフォレスト　インスタンス作成
rfc = RandomForestClassifier(n_estimators=100, random_state=1)

# モデル訓練
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [13]:
# テストデータから推測
rfc_pred = rfc.predict(X_test)

# 混合行列を表示
print(confusion_matrix(y_test, rfc_pred))

[[ 961    0    3    0    0    1    6    1   10    1]
 [   0 1126    6    5    0    1    0    6    5    3]
 [   5    2  922    4    9    0    7    9    8    1]
 [   2    4   18  953    1   15    3   13   13   12]
 [   1    0    1    0  848    2   12    3    4   35]
 [   4    5    1   31    2  869    9    1   10    5]
 [   8    3    0    0    3   10  935    0    2    0]
 [   4    5   17    0    9    1    0 1003    4   12]
 [   3   10   11   22    6   10    5    7  884   11]
 [   5    5    1   15   23    5    4   14    4  960]]


In [14]:
# 正解率を表示
print(accuracy_score(y_test, rfc_pred))

0.9461


テキストとは異なりますが、最も高い精度が出ました。

# ここから CNN 。の前に、基礎を固めることにします。

ここから先は、CNN になるので、以下の学習を済ませようと思います。

1. [線形代数 入門](https://student.codexa.net/contents/index/6)
2. [統計入門（前編）](https://student.codexa.net/contents/index/8)
3. [統計入門（後編）](https://student.codexa.net/contents/index/9)
4. [Numpy 入門](https://student.codexa.net/contents/index/1)
5. [Pandas 入門](https://student.codexa.net/contents/index/2)
6. [Matplotlib 入門](https://student.codexa.net/contents/index/7)
7. [線形回帰 入門](https://student.codexa.net/contents/index/10)
8. [実践 線形回帰](https://student.codexa.net/contents/index/11)
9. [実践 ロジスティック回帰](https://student.codexa.net/contents/index/12)
10. [決定木とランダムフォレスト](https://student.codexa.net/contents/index/13)
11. [サポートベクターマシン](https://student.codexa.net/contents/index/14)

# そして上記の受講を完了しました！